In [ ]:
import itertools as it

import dask
import dask.array as da
import numpy as np
import xarray as xr

In [ ]:
@dask.delayed
def tile_function(i, j):
    return np.ones((10, 20)) * i * j

In [ ]:
arr = da.concatenate([
    da.from_delayed(tile_function(i, j), (10, 20), dtype=float)
    for i, j in it.product(range(10), range(20))
], axis=-1)
arr

In [ ]:
xarr = xr.DataArray(arr, coords={"x": range(arr.shape[0]), "y": range(arr.shape[1])})

In [ ]:
xarr2 = xarr.compute()

In [ ]:
xarr2.values

In [ ]:
xarr